In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import numpy as np
import evaluate

In [2]:
# --- 1. CARREGAR O MELHOR MODELO E O CONJUNTO DE TESTE FINAL ---

# Carregar o nosso melhor modelo (v2)
model_path = "/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_v2_final"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Carregar o conjunto de teste que guardamos
path_final_test_set = '/Users/giossaurus/Developer/leia_tcc/data/processed/final_test_set.csv'
df_final_test = pd.read_csv(path_final_test_set)

# Verificar se a coluna 'question' existe ou se já é 'text'
if 'question' in df_final_test.columns:
    df_final_test = df_final_test.rename(columns={'question': 'text'})

# Se o label ainda estiver em formato de texto, precisamos mapear para números
# Carregar os labels do seed set para manter consistência
df_seed_anotado = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv')
labels = df_seed_anotado['intent_choice'].unique().tolist()
label2id = {label: i for i, label in enumerate(labels)}

# Verificar se precisa mapear labels
if df_final_test['label'].dtype == 'object':
    df_final_test['label'] = df_final_test['label'].map(label2id)

final_test_dataset = Dataset.from_pandas(df_final_test)

print(f"Carregado melhor modelo de '{model_path}'.")
print(f"Carregado conjunto de teste final com {len(final_test_dataset)} exemplos.")
print(f"Labels únicos no teste: {sorted(df_final_test['label'].unique())}")

Carregado melhor modelo de '/Users/giossaurus/Developer/leia_tcc/models/leia_classifier_v2_final'.
Carregado conjunto de teste final com 40 exemplos.
Labels únicos no teste: [np.int64(0), np.int64(1), np.int64(2), np.int64(3)]


In [3]:
# --- 2. PREPARAR OS DADOS DE TESTE ---

# Tokenizar o conjunto de teste
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_final_test_dataset = final_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [4]:
# --- 3. EXECUTAR A AVALIAÇÃO FINAL ---

# Carregar as métricas
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_final_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="weighted"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="weighted"))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="weighted"))
    
    return results

# Criar um Trainer apenas para a avaliação
final_trainer = Trainer(
    model=model,
    compute_metrics=compute_final_metrics
)

# Fazer as predições e obter as métricas
final_evaluation_results = final_trainer.evaluate(eval_dataset=tokenized_final_test_dataset)

print("\n--- RESULTADOS FINAIS NO CONJUNTO DE TESTE ---")
print(f"Acurácia Final: {final_evaluation_results['eval_accuracy']:.4f}")
print(f"F1-Score Final (Ponderado): {final_evaluation_results['eval_f1']:.4f}")
print(f"Precisão Final (Ponderada): {final_evaluation_results['eval_precision']:.4f}")
print(f"Recall Final (Ponderado): {final_evaluation_results['eval_recall']:.4f}")
print("-------------------------------------------------")

/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



--- RESULTADOS FINAIS NO CONJUNTO DE TESTE ---
Acurácia Final: 0.5000
F1-Score Final (Ponderado): 0.4693
Precisão Final (Ponderada): 0.4492
Recall Final (Ponderado): 0.5000
-------------------------------------------------


/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
